# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Lê Quốc Trung
- MSSV: 20127369
- Lớp: 20clc11

# Import

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import *
from operator import itemgetter

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [14]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa 10 đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa 10 đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

#x,y cần chuyển về cho phù hợp với numpy
#lr.fit(x_train, y_train)
#lr.get_params -> w1, w2,...
#lr.predict(x_test) ->y_test_predict
# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [15]:
class OLSLinearRegression:
    def fit(self, x, y):   
        X_pinv = np.linalg.inv(x.T @ x) @ x.T    
        self.w = X_pinv @ y
        return self

    def get_params(self):
        return self.w

    def predict(self, x):
        return np.sum(self.w.ravel() * x, axis=1)


def LinearRegression(x_train, y_train, x_test, y_test):
    x_train = np.array(x_train)
    if x_train.shape.__len__() == 1:
        x_train = x_train.reshape(-1, 1)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    if x_test.shape.__len__() == 1:
        x_test = x_test.reshape(-1, 1)
    y_test = np.array(y_test)

    lr = OLSLinearRegression().fit(x_train, y_train)
    
    y_hat = lr.predict(x_test)
    x_hat = lr.get_params().reshape(-1)
    RMSE = np.sqrt(np.mean((y_test.ravel() - y_hat.ravel())**2))
    
    
    return x_hat, RMSE
    

# Yêu cầu 1a: Sử dụng toàn bộ 10 đặc trưng đề bài cung cấp (2 điểm) 

In [16]:
# Phần code cho yêu cầu 1a
x_hat, rmse = LinearRegression(X_train, y_train, X_test, y_test)
    
print(x_hat)

[ 1.51013627e-02  9.02199807e-02  4.29218175e-02  1.39289117e-01
 -5.67332827e-01 -1.00765115e-04  7.40713438e-01  1.90935798e-01
  2.45059736e+01  2.39351661e+00]


In [17]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
print(rmse)

7.0640464305843516


Công thức hồi quy

$$\text{Life expectancy} = 1.5e-02*Adult Mortality + 9.02e-02*BMI + 4.29e-02Polio + 1.39e-01*Diphtheria - 5.6e-01*IV/AIDS - 1.007e-04*GDP + 7.4e-01*Thinness age 10-19 + 1.9e-01*Thinness age 5-9 + 2.4e+01*Income composition of resources + 2.39e+00*Schooling$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng, tìm mô hình cho kết quả tốt nhất (2 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [18]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu
column = ["Adult Mortality", "BMI", "Polio", "Diphtheria", "HIV/AIDS", "GDP", "Thinness age 10-19", "Thinness age 5-9", "Income composition of resources", "Schooling", "Life expectancy"]
feature = [each for each in column if each != 'Life expectancy']
def KFold_CrossValidation(data, feature, k=5):
    
    col = data.shape[1] - 1
    x_hat = np.zeros(col)
    data_rmse = np.zeros(col)
    
    kf = KFold(n_splits=k, shuffle=True)
    
    for train_id, test_id in kf.split(data):
        train_data = data.iloc[train_id]
        test_data = data.iloc[test_id]
        i = 0
        for each in feature:
            temp_train = train_data.loc[:, [each, 'Life expectancy']]
            feature_train = temp_train.loc[:,[each]]
            label_train = temp_train.loc[:,['Life expectancy']]
            train_x, train_y = feature_train, label_train

            temp_test = test_data.loc[:, [each, 'Life expectancy']]
            feature_test = temp_test.loc[:,[each]]
            label_test = temp_test.loc[:,['Life expectancy']]
            test_x, test_y = feature_test, label_test
            
            x_hat_lr, rmse = LinearRegression(train_x, train_y, test_x, test_y)
            
            x_hat[i] += x_hat_lr
            data_rmse[i] +=rmse
            i += 1  
        
    x_hat /= 5
    data_rmse /= 5
    
    best_feature_index = min(enumerate(data_rmse), key=itemgetter(1))[0]
    
    
    return x_hat, data_rmse, best_feature_index


x_hat_list, rmse_list, best_feature = KFold_CrossValidation(train, feature)
print("best feature: ", column[best_feature], rmse_list[best_feature])
print(x_hat_list)
print(rmse_list)
print(best_feature)

best feature:  Schooling 11.7913645544782
[2.47811218e-01 1.50570692e+00 7.85639969e-01 7.84449481e-01
 3.59956670e+00 3.02707761e-03 6.93121801e+00 6.88324411e+00
 1.04706343e+02 5.55751915e+00]
[46.22171015 27.90662293 18.00479374 16.01299507 67.12203248 60.22180982
 51.82214495 51.73701657 13.33212111 11.79136455]
9


In [19]:
# Huấn luyện lại mô hình best_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
x_hat, rmse = LinearRegression(X_train[column[best_feature]], y_train, X_test[column[best_feature]], y_test)

print(x_hat)


[5.5573994]


In [20]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_feature_model

print(rmse)

10.26095039165537


Công thức hồi quy

$$\text{Life expectancy} = 5.55 * Schooling$$

# Yêu cầu 1c: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

In [21]:
# Phần code cho yêu cầu 1c
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu
def process_data1(train):
    data = train.loc[:, ["Adult Mortality", "BMI", "Polio", "Diphtheria", "HIV/AIDS", "GDP", "Thinness age 10-19", "Thinness age 5-9", "Income composition of resources", "Schooling", "Life expectancy"]]
    data['Model 1'] = data.iloc[:, :-1].sum(axis=1)
    return data.loc[:, ['Model 1', 'Life expectancy']]

def process_data2(train):
    data = train.loc[:, ["Diphtheria", "Income composition of resources", "Schooling", 'Life expectancy']]
    data['Model 2'] = data.iloc[:, :-1].sum(axis=1)
    return data.loc[:, ['Model 2', 'Life expectancy']]

def process_data3(train):
    data = train.loc[:, ['GDP', 'Life expectancy']]
    data['Model 3'] = data['GDP']
    return data.loc[:, ['Model 3', 'Life expectancy']]

def process_data4(train):
    data = train.loc[:, ['GDP','Life expectancy']]
    data['Model 4'] = np.sqrt(data['GDP'])
    return data.loc[:, ['Model 4', 'Life expectancy']]

In [22]:
# Phần code cho yêu cầu 1c
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu
def process_data1(train):
    data = train.loc[:, ["Adult Mortality", "BMI", "Polio", "Diphtheria", "HIV/AIDS", "GDP", "Thinness age 10-19", "Thinness age 5-9", "Income composition of resources", "Schooling", "Life expectancy"]]
    data['Model 1'] = data.iloc[:, :-1].sum(axis=1)
    return data.loc[:, ['Model 1', 'Life expectancy']]

def process_data2(train):
    data = train.loc[:, ["Diphtheria", "Income composition of resources", "Schooling", 'Life expectancy']]
    data['Model 2'] = data.iloc[:, :-1].sum(axis=1)
    return data.loc[:, ['Model 2', 'Life expectancy']]

def process_data3(train):
    data = train.loc[:, ['GDP', 'Life expectancy']]
    data['Model 3'] = data['GDP']
    return data.loc[:, ['Model 3', 'Life expectancy']]

def process_data4(train):
    data = train.loc[:, ['GDP','Life expectancy']]
    data['Model 4'] = np.sqrt(data['GDP'])
    return data.loc[:, ['Model 4', 'Life expectancy']]

def process_data5(train):
    data = train.loc[:, ['GDP', 'Life expectancy']]
    data['Model 5'] = data['GDP']**2
    return data.loc[:, ['Model 5', 'Life expectancy']]

def process_data6(train):
    data = train.loc[:, ['Schooling', 'Life expectancy']]
    data['Model 6'] = np.sqrt(data['Schooling'])
    return data.loc[:, ['Model 6', 'Life expectancy']]

def process_data7(train):
    data = train.loc[:, ['Schooling', 'Life expectancy']]
    data['Model 7'] = data['Schooling']**2
    return data.loc[:, ['Model 7', 'Life expectancy']]


# sum all feature
data1 = process_data1(train)
                     
# sum of 3 best feature
data2 = process_data2(train)
                     
# choose 1 bad feature (high rmse)
data3 = process_data3(train)
                     
# sqrt(data3)
data4 = process_data4(train)
                     
# square(data3)
data5 = process_data5(train)
                     
# sqrt(best_feature)
data6 = process_data6(train)
                     
# square(best_feature)
data7 = process_data7(train)

x_hat1, rmse1, temp = KFold_CrossValidation(data1, ['Model 1'])
x_hat2, rmse2, temp = KFold_CrossValidation(data2, ['Model 2'])
x_hat3, rmse3, temp = KFold_CrossValidation(data3, ['Model 3'])
x_hat4, rmse4, temp = KFold_CrossValidation(data4, ['Model 4'])
x_hat5, rmse5, temp = KFold_CrossValidation(data5, ['Model 5'])
x_hat6, rmse6, temp = KFold_CrossValidation(data6, ['Model 6'])
x_hat7, rmse7, temp = KFold_CrossValidation(data7, ['Model 7'])

print(rmse1, rmse2, rmse3, rmse4, rmse5, rmse6, rmse7, sep='\n')




[59.13710468]
[14.20886525]
[60.20071271]
[43.79480973]
[66.15172228]
[6.37130943]
[24.42182241]


In [23]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
my_best_model_test = test.loc[:, ['Schooling']]
my_best_model_test['Model 6'] = np.sqrt(my_best_model_test['Schooling'])
#rint(my_best_model_test['Model 6'])
#print()
my_best_model_x, my_best_model_rmse = LinearRegression(data6['Model 6'], y_train, my_best_model_test['Model 6'], y_test)
print(my_best_model_x)

[20.02508698]


In [24]:
# Gọi hàm RMSE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
print(my_best_model_rmse)

5.318134128327009


Công thức hồi quy

$$\text{Life expectancy} = 20.02 * sqrt(Schooling)$$